In [ ]:
from __future__ import division, print_function

import numpy as np
import matplotlib.pyplot as plt

from corner import corner
from acor import acor
from UL_uncert import UL_uncert

%matplotlib inline

In [ ]:
ephem = 'DE421'
rundir = '/home/pbaker/nanograv/bwm/tests/fixsky/'

In [ ]:
# this changes depending on which "names" are used in enterprise....

with open(rundir + 'params.txt', 'r') as f:
    params = [line.rstrip('\n') for line in f]

params = params[-3:]
idx_A = 0
idx_t0 = 2

In [ ]:
chain_raw = np.loadtxt(rundir + 'chain_1.txt')
burn = int(0.1 * len(chain_raw))
chain = chain_raw[burn:]    
chain_bwm = chain[:,-7:-4]  # no ephem model

# de-log x-axis for ULs
#chain_bwm[:,idx_A] = 10**chain_bwm[:,idx_A]
#params[idx_A] = r'$A_\mathrm{BwM}$'
    
corr = acor(chain_bwm[:,idx_A])[0]
N = len(chain_bwm)
print(ephem)
print("N = {}, corr = {}, Neff = {}".format(N, corr, N/corr))

In [ ]:
corner_kwargs = {'bins':30,
                 'labels':params,
                 'smooth':1.0,
                 'plot_datapoints':True,
                 'plot_density':False,
                 'plot_contours':False,
                 'fill_contours':False,}

fig = corner(chain_bwm, color='C0', **corner_kwargs);

In [ ]:
ax = []
fig = plt.figure(figsize=(20,12))

for pp, par in enumerate(params):
    ax.append(fig.add_subplot(2,3,pp+1))
    ax[pp].hist(chain_bwm[:,pp], 30, normed=True, histtype='step', label=ephem)
    ax[pp].set_xlabel(par)

ax[0].legend(loc='upper left')
ax[idx_A].set_yscale('log')
ax[idx_t0].set_yscale('log')

In [ ]:
from matplotlib.colors import LogNorm
lognorm = LogNorm()

hist_kwargs = {'bins':50,
               'normed':True,
               'cmap':'YlOrRd',
               #'vmin':1.0e5,
               #'vmax':1.0e10,
               'norm':lognorm,
                }

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)

t0_ch = chain_bwm[:,idx_t0]
if max(chain_bwm[:,idx_A])>0:
    h_ch = np.log10(chain_bwm[:,idx_A])
else:
    h_ch = chain_bwm[:,idx_A]
    
ax.hist2d(t0_ch, h_ch, **hist_kwargs)
ax.set_xlabel(r'$t_0$ (MJD)')
ax.set_ylabel(r'$\log_{10}(A_{BWM})$')
ax.set_ylim([-18,-11])